In [30]:
import pandas
from pandas import DataFrame
%matplotlib inline
import seaborn
from sklearn.ensemble import RandomForestRegressor

from sklearn import cross_validation


In [2]:
train_df = DataFrame.from_csv("train.csv", index_col=['Store', 'Date'], parse_dates=['Date'])
train_df.head()

/opt/conda/lib/python3.4/site-packages/pandas/io/parsers.py:1170: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  data = self._reader.read(nrows)


,,DayOfWeek,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
Store,Date,,,,,,,
1,2015-07-31,5,5263,555,1,1,0,1
2,2015-07-31,5,6064,625,1,1,0,1
3,2015-07-31,5,8314,821,1,1,0,1
4,2015-07-31,5,13995,1498,1,1,0,1
5,2015-07-31,5,4822,559,1,1,0,1


In [23]:
regressor = RandomForestRegressor(n_estimators=25)

In [5]:
train_df.reset_index(inplace=True)
features = [col for col in train_df.columns if col != 'Sales']

In [6]:
score = cross_val_score(regressor, train_df[features], train_df['Sales']).mean()

ValueError: could not convert string to float: 'a'

In [7]:
def numericStateHoliday(x):
    if x == 'a':
        return 1
    elif x == 'b':
        return 2
    elif x == 'c':
        return 3
    else:
        return x
train_df['StateHoliday'] = train_df['StateHoliday'].map(lambda x: numericStateHoliday(x))

In [8]:
score = cross_val_score(regressor, train_df[features], train_df['Sales']).mean()

TypeError: float() argument must be a string or a number, not 'Timestamp'

In [20]:

train_df['Date'] = train_df['Date'].map(lambda x: x.value)

In [24]:
%%time 
score = cross_val_score(regressor, train_df[features], train_df['Sales'], n_jobs=-1).mean()

CPU times: user 768 ms, sys: 812 ms, total: 1.58 s
Wall time: 2min 8s


In [25]:
score

0.9288719039257386

In [26]:
%%time 
features = [x for x in features if x != 'Date']
score = cr(regressor, train_df[features], train_df['Sales'], n_jobs=-1).mean()

CPU times: user 692 ms, sys: 684 ms, total: 1.38 s
Wall time: 1min 53s


In [62]:
regressor = RandomForestRegressor(n_estimators=25)
predict = cross_validation.cross_val_predict(regressor, train_df[features], train_df['Sales'], n_jobs=-1)

In [35]:
len(predict)

1017209

In [37]:
len(train_df['Sales'])

1017209

In [41]:
import math
math.pow(3, 2)

9.0

In [55]:
score_df = train_df[['Sales']]
score_df['Predict'] = predict
score_df = score_df[score_df['Sales'] != 0]

/opt/conda/lib/python3.4/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [56]:
score_df['PercentError'] =  (score_df['Sales'] - score_df['Predict']) / score_df['Sales']

In [57]:
score_df['PercentErrorSquared'] = score_df['PercentError'] * score_df['PercentError']

In [58]:
math.sqrt(score_df['PercentErrorSquared'].mean())

0.025645641555457104

In [76]:
math.sqrt(0.025645641555457104)

0.16014256634467022